<a href="https://colab.research.google.com/github/OlyKoek/Udemy-AIperfectMaster-colabo/blob/main/embeding_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec and doc2vec学習

## Sentence-Transformer体験(doc2vec系)

In [2]:
# --- Colab 初期設定 ---
!pip install -q sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.5 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [69]:
# モデル選択（日本語含む多言語対応）
# SentenceTransformer モデルをロードします。
# "paraphrase-multilingual-MiniLM-L12-v2" は、多くの言語に対応した比較的小さなモデルです。
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [70]:
# テスト語彙
# 埋め込みベクトルを取得したい単語のリストを定義します。
# words = ["猫", "犬", "人間", "先生", "学生", "こころ", "恋", "友達"]
words = ["王", "エリザベス","男","女","女性","おじいさん","おばあさん","妻","夫"]

# 埋め込みを取得
# 定義した単語リストの埋め込みベクトルを取得します。
# convert_to_numpy=True で結果を NumPy 配列として取得します。
# normalize_embeddings=True で埋め込みベクトルを正規化し、コサイン類似度の計算を容易にします。
embeddings = model.encode(words, convert_to_numpy=True, normalize_embeddings=True)
print(f"単語数: {len(words)}")
print(f"埋め込み数: {len(embeddings[0])}") # 各単語の埋め込みベクトルの次元数

単語数: 9
埋め込み数: 384


In [71]:
# コサイン類似度計算関数
# 指定した単語に最も類似する単語を見つける関数を定義します。
def most_similar(word, topn=10):
    # 指定した単語がリストに存在するか確認します。
    if word not in words:
        print(f"{word} はリストに存在しません。")
        return
    # 指定した単語のインデックスを取得します。
    idx = words.index(word)
    # 指定した単語の埋め込みベクトルと他の全ての単語の埋め込みベクトルとのコサイン類似度を計算します。
    sims = util.cos_sim(embeddings[idx], embeddings)[0].cpu().numpy()
    # 類似度が高い順にインデックスをソートします。
    sorted_idx = np.argsort(-sims)
    # 結果を表示します。
    print(f"\n🔎 「{word}」に類似する単語:")
    # 類似度が高い順に上位 topn 個の単語と類似度を表示します。（最初の要素は自分自身なのでスキップします）
    for i in sorted_idx[1:topn+1]:
        print(f"  {words[i]} : 類似度 {sims[i]:.3f}")

# 試してみる
# most_similar("吾輩") # コメントアウトされています
most_similar("王")
# most_similar("先生") # コメントアウトされています


🔎 「王」に類似する単語:
  男 : 類似度 0.755
  夫 : 類似度 0.750
  おじいさん : 類似度 0.689
  エリザベス : 類似度 0.560
  おばあさん : 類似度 0.480
  妻 : 類似度 0.451
  女 : 類似度 0.432
  女性 : 類似度 0.338


### 少し長い文章同士の比較

In [82]:
# 比較したい日本語の文章リストを定義します。
sentences = [
    "猫は可愛い動物です。",
    "犬は忠実なペットです。",
    "動物園には様々な動物がいます。",
    "今日の天気は晴れです。",
    "この本はとても面白いです。",
    "あの猫はとても可愛いです。",
    "彼の犬はとても賢いです。",
]

# 定義した文章リストに対して、SentenceTransformer モデルを使用して埋め込みベクトルを計算します。
# convert_to_numpy=True で結果を NumPy 配列として取得し、normalize_embeddings=True で正規化します。
sentence_embeddings = model.encode(sentences, convert_to_numpy=True, normalize_embeddings=True)

print(f"文章数: {len(sentences)}")
print(f"文章埋め込み数: {len(sentence_embeddings[0])}") # 各文章の埋め込みベクトルの次元数

# 計算された文章埋め込みベクトル間のコサイン類似度を計算します。
# util.cos_sim 関数は、2つの埋め込みセット間のコサイン類似度行列を計算します。
similarity_matrix = util.cos_sim(sentence_embeddings, sentence_embeddings).cpu().numpy()

# 計算された類似度行列を表示します。
# DataFrame を使用すると、どの文章ペアの類似度が高いかが見やすくなります。
import pandas as pd

similarity_df = pd.DataFrame(similarity_matrix, index=sentences, columns=sentences)

print("\n文章間のコサイン類似度行列:")
display(similarity_df)

文章数: 7
文章埋め込み数: 384

文章間のコサイン類似度行列:


,猫は可愛い動物です。,犬は忠実なペットです。,動物園には様々な動物がいます。,今日の天気は晴れです。,この本はとても面白いです。,あの猫はとても可愛いです。,彼の犬はとても賢いです。
猫は可愛い動物です。,1.000000,0.296190,0.331150,0.080973,0.165545,0.931925,0.247535
犬は忠実なペットです。,0.296190,1.000000,0.379999,0.208162,0.138141,0.190688,0.644293
動物園には様々な動物がいます。,0.331150,0.379999,1.000000,0.100005,0.134910,0.190325,0.256741
今日の天気は晴れです。,0.080973,0.208162,0.100005,1.000000,0.054622,0.089441,0.091673
この本はとても面白いです。,0.165545,0.138141,0.134910,0.054622,1.000000,0.209227,0.265587
あの猫はとても可愛いです。,0.931925,0.190688,0.190325,0.089441,0.209227,1.000000,0.241906
彼の犬はとても賢いです。,0.247535,0.644293,0.256741,0.091673,0.265587,0.241906,1.000000


## FastT体験 (word2vec系)

In [72]:
# bpemb ライブラリをインストールします。これは FastText の事前学習済みモデルを簡単に利用するためのライブラリです。
!pip install bpemb
# BPEmb クラスをインポートします。
from bpemb import BPEmb

In [73]:
# 日本語の BPEmb モデルをロードします。
# lang="ja" で日本語を指定し、dim=300 で埋め込みベクトルの次元数を300に指定します。
bpemb_ja = BPEmb(lang="ja", dim=300)
# "猫" という単語の埋め込みベクトルを取得して表示します。
bpemb_ja.embed("猫")

array([[ 0.007944,  0.177064,  0.092667,  0.494488,  0.186388, -0.037287,
        -0.387206,  0.048648,  0.175421, -0.205781,  0.241435, -0.035705,
        -0.109678, -0.198312, -0.118789, -0.156208,  0.056966,  0.236318,
         0.076374, -0.165804,  0.483365, -0.172973,  0.172958,  0.127344,
        -0.124194,  0.166179,  0.146671, -0.265863,  0.096292, -0.076168,
        -0.119885,  0.360599,  0.040988,  0.028406, -0.028427,  0.034044,
        -0.116466,  0.027716, -0.204683,  0.129862, -0.172068,  0.347782,
        -0.162706, -0.08533 , -0.123627,  0.015179,  0.126716, -0.270145,
         0.220954,  0.067894, -0.510218, -0.152041,  0.4393  , -0.335511,
        -0.226223, -0.160528,  0.232765,  0.044341, -0.175786,  0.116366,
        -0.162253,  0.002957, -0.482127, -0.181595,  0.259897, -0.023736,
         0.278957,  0.010745, -0.113892,  0.174106,  0.221188, -0.090708,
         0.018256, -0.133169, -0.043962, -0.052177, -0.089778, -0.021125,
        -0.049194,  0.608331,  0.04216

In [74]:
# "吾輩は" という単語列に最も類似する単語を取得します。
# BPEmb は単語をサブワードに分割して処理します。
bpemb_ja.most_similar(["吾","輩","は"])

[('藤', 0.49648740887641907),
 ('也', 0.4790944755077362),
 ('哉', 0.4779980182647705),
 ('郎', 0.4708433151245117),
 ('圭', 0.4689044952392578),
 ('兄', 0.4665144681930542),
 ('一郎', 0.4663138687610626),
 ('二郎', 0.46086856722831726),
 ('太郎', 0.44819310307502747),
 ('山本', 0.4473738670349121)]

In [75]:
# 単語アナロジーを計算します。
# positive には類似度を加えたい単語、negative には類似度を減らしたい単語を指定します。
# この例では、「王」と「男」のベクトルを足し、「強」のベクトルを引いた結果に最も近い単語を検索しています。
bpemb_ja.most_similar(positive=["王","男"], negative=["強"])

[('次男', 0.5161638855934143),
 ('女', 0.5138992667198181),
 ('長男', 0.5130579471588135),
 ('の娘', 0.5081402659416199),
 ('夫', 0.49944910407066345),
 ('孫', 0.4961977005004883),
 ('の子', 0.48964518308639526),
 ('息子', 0.4835880994796753),
 ('甥', 0.47327569127082825),
 ('妃', 0.4690869450569153)]

## Fasttext でoos (out-of-vocabulary) 単語を扱う例を追加する

### Subtask:
FastText が未知語（OOV）をどのように扱えるかを示す新しいコードセルとその説明を追加します。


**Reasoning**:
The previous code block failed because the `bpemb_ja` object does not have a `vector()` method. The correct way to get the combined vector for a word (including OOV words) is to get the subword embeddings using `embed()` and then sum or average them. Let's fix the code to reflect this.



In [88]:
# OOV 単語を定義します。この単語は学習データに含まれていない可能性が高いです。
oov_word = "ピヨピヨ語" # 例：存在しない可能性の高い架空の単語

# 定義した未知語に対して、BPEmb モデルを使用して埋め込みベクトルを取得します。
# FastText のサブワードベースのアプローチにより、未知語でも埋め込みが生成されます。
# bpemb_ja.embed() はサブワードごとの埋め込みのリストを返します。
oov_embeddings = bpemb_ja.embed(oov_word)

print(f"未知語「{oov_word}」のサブワード埋め込み（次元数: {len(oov_embeddings[0])}）:")
# 埋め込みベクトルを表示します。
display(oov_embeddings)

# 未知語の合計ベクトルは、その単語を構成するサブワードの埋め込みを合計することで得られます。
# この合計ベクトルが、多くの場合、単語全体の埋め込みとして使用されます。
oov_vector = np.sum(oov_embeddings, axis=0)
print(f"\n未知語「{oov_word}」の合計ベクトル（単語埋め込みとして使用可能）:")
display(oov_vector)

# さらに、未知語がどのようにサブワードに分割されたかを確認することもできます。
subwords = bpemb_ja.encode(oov_word)
print(f"\n未知語「{oov_word}」のサブワード分割:")
display(subwords)

bpemb_ja.most_similar(positive=[oov_vector])

未知語「ピヨピヨ語」のサブワード埋め込み（次元数: 300）:


array([[ 0.258476,  0.236887, -0.276728, ...,  0.3263  ,  0.219412,
         0.078539],
       [-0.064116, -0.091526,  0.122248, ..., -0.035749, -0.382148,
        -0.154039],
       [ 0.292409,  0.161463, -0.159357, ...,  0.185985, -0.108042,
         0.047171],
       [-0.064116, -0.091526,  0.122248, ..., -0.035749, -0.382148,
        -0.154039],
       [-0.772598, -0.312042, -0.35815 , ...,  0.270523, -0.017078,
         0.068344]], dtype=float32)


未知語「ピヨピヨ語」の合計ベクトル（単語埋め込みとして使用可能）:


array([-3.49945039e-01, -9.67440009e-02, -5.49739003e-01,  1.26907802e+00,
       -2.22300991e-01, -3.06234956e-01,  2.34531984e-01, -2.88133979e-01,
        7.32806027e-01, -1.31751704e+00,  6.25710338e-02, -3.41412008e-01,
        3.52215976e-01, -1.82561010e-01,  4.31226015e-01,  2.91187018e-01,
       -1.09850693e+00,  7.85420090e-02,  2.34388992e-01,  9.84820127e-02,
       -7.11636007e-01, -4.23234969e-01,  4.52462018e-01,  6.56556010e-01,
       -3.16110015e-01,  2.19500989e-01, -4.24512982e-01, -1.22740197e+00,
        7.84819722e-02,  2.94966012e-01,  1.51823997e+00,  4.18935984e-01,
       -5.54779768e-02, -6.60143018e-01,  5.51672995e-01,  1.15239993e-02,
       -3.96295011e-01,  1.94541299e+00,  6.27195001e-01,  2.09058970e-01,
       -1.16730201e+00,  1.54673493e+00,  3.22730094e-02,  3.40880007e-02,
        2.15019703e+00, -1.29847503e+00,  3.77983004e-01,  1.62907600e+00,
        6.31637990e-01, -2.81626606e+00, -4.56191003e-01, -9.43914950e-01,
        9.36673999e-01, -


未知語「ピヨピヨ語」のサブワード分割:


['▁ピ', 'ヨ', 'ピ', 'ヨ', '語']

[('ヨ', 0.7970728874206543),
 ('イ', 0.7242064476013184),
 ('ピ', 0.7022724747657776),
 ('ヌ', 0.690114438533783),
 ('ル', 0.6893051266670227),
 ('ナ', 0.6878511905670166),
 ('ラ', 0.6846035122871399),
 ('テ', 0.6797680258750916),
 ('ッ', 0.6780124306678772),
 ('カ', 0.6755659580230713)]